In [1]:
from gensim.models import Word2Vec

model = Word2Vec.load("300features_40minwords_10context")

In [2]:
import pandas as  pd

train = pd.read_csv("labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("testData.tsv",header=0,delimiter="\t",quoting=3)

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    words = review_text.lower().split()
    if remove_stopwords==True:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return words

num_features=300
min_words_count =40
downsampling=1e-3
context= 10
num_workers = 4


In [3]:
import numpy as np
def makeFeatureVec(words,model,num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec,model.wv[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [4]:
def getAvgFeatureVecs(reviews,model,num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        if counter%1000==0:
            print ("review %d of %d" %(counter,len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review,model,num_features)
        counter+=1
    return reviewFeatureVecs

In [5]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews,model,num_features)

print("创建测试集评论的平均特征向量")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews,model,num_features)

/var/folders/9c/fs4ghgt579d29fb6jsxdrls40000gn/T/ipykernel_28393/4243584205.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


review 0 of 25000
review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
review 20000 of 25000
review 21000 of 25000
review 22000 of 25000
review 23000 of 25000
review 24000 of 25000
创建测试集评论的平均特征向量
review 0 of 25000
review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
review 20000 of 25000

In [6]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)

print("将labeled training data拟合到随机森林...")
forest = forest.fit(trainDataVecs,train["sentiment"])

result = forest.predict(testDataVecs)
output = pd.DataFrame( data={"Id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

将labeled training data拟合到随机森林...
